In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
from matplotlib import pyplot as plt
import csv

In [ ]:
assay_file = '../data/assay_matrix_discrete_270_assays.csv' # the same assay matrix, where blanks are replaced with -1
assay_df = pd.read_csv(assay_file).fillna(-1)
assay_header = assay_df.columns[1:]
assay_np = assay_df.to_numpy()[:, 1:].astype('int64')

In [ ]:
assay_np_true = np.where(assay_np == 1,1,0)
intersections_relative = np.zeros((270,270))
assay_activations = []
for i in range(assay_np_true.shape[1]):
    assay_activations.append(set(np.argwhere(assay_np_true[:,i]==1).flatten()))
    
for i in range(270):
    for j in range(270):
        if len(assay_activations[i]) != 0 and len(assay_activations[j]) != 0:
            intersections_relative[i,j] = len(assay_activations[i].intersection(assay_activations[j])) / len(assay_activations[i].union(assay_activations[j]))
            intersections_relative[j,i] = len(assay_activations[i].intersection(assay_activations[j])) / len(assay_activations[i].union(assay_activations[j]))
        else:
            intersections_relative[i,j] = 0
            intersections_relative[j,i] = 0
        
        
plt.gcf().set_size_inches(13, 13)    
pl = sb.clustermap(intersections_relative, metric = 'cosine')

In [ ]:
assay_header = list(assay_header)
modalities = ['cp', 'mobc','ge']
palette = {0: 'red', 1: 'blue', 2: 'blue', 3: 'blue', 4: 'blue', 5: 'blue'}
auc_to_measure = 'auc_90'
for modality in modalities:
    assay_counts = {}
    for i in range(5):
        pred_final_file = '../predictions/chemical_cv{}/2022_01_evaluation_all_data_python.csv'.format(i)
        with open(pred_final_file) as f:
            reader = csv.reader(f)
            measure = next(reader).index(auc_to_measure)
            for row in reader:
                if row[2] == '{}_es_op'.format(modality):
                    if row[0] not in assay_counts.keys() and row[measure] == 'TRUE':
                        assay_counts[row[0]] = 1
                    elif row[0] in assay_counts.keys() and row[measure] == 'TRUE':
                        assay_counts[row[0]] =  assay_counts[row[0]] + 1
                    elif row[0] not in assay_counts.keys() and row[measure] != 'TRUE':
                        assay_counts[row[0]] = 0
    

    counts_matrix = np.empty(270, dtype=object)
    for key in assay_counts.keys():
        counts_matrix[assay_header.index(key)] = assay_counts[key]
                
    intersections_relative_df = pd.DataFrame(data=intersections_relative)
    counts_matrix_df = pd.DataFrame(data=counts_matrix)
    intersections_relative_counts_matrix_df = pd.concat([counts_matrix_df, intersections_relative_df], axis=1)
    intersections_relative_counts_matrix_df.columns = ['auc09'] + assay_header
    row_colors = intersections_relative_counts_matrix_df.auc09.map(palette)
    print(modality, list(row_colors).count('blue'))
    sb.clustermap(intersections_relative_counts_matrix_df.iloc[:,1:], metric="cosine", row_colors=row_colors)
    plt.show()